# graph-nd
__Knowledge in Graphs not Documents!__

1. GraphRAG in 4 lines of code in 5 minutes. No Cypher necessary.
2. Designed to extend to production - not just a demo.
3. Easily merges mixed structured & unstructured data.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('.env', override=True)

uri = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

In [3]:
import os

from graphrag import  GraphRAG
from neo4j import GraphDatabase
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

db_client = GraphDatabase.driver(uri, auth=(username, password))
embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002')
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)


# Instantiate graph
graphrag = GraphRAG(db_client, llm, embedding_model)

# 1) Get the graph schema. Can also define exactly via json/pydantic spec
graphrag.schema.infer("a simple graph of hardware components "
                      "where components (with id, name, and description properties)  "
                      "can be types of or inputs to other components.")

# 2) Merge data. Can also directly merge node & rel records extracted else where
graphrag.data.merge_csvs(['component-types.csv', 'component-input-output.csv']) # structured data
graphrag.data.merge_pdf('component-catalog.pdf') #unstructured

# 3) GraphRAG agent for better answers.
graphrag.agent("what sequence of components depend on silicon wafers?")

[Schema] Generated schema:
 {
    "description": "A simple graph schema for hardware components and their relationships.",
    "nodes": [
        {
            "description": "Represents a hardware component with an id, name, and description.",
            "id": {
                "description": "",
                "name": "id",
                "type": "STRING"
            },
            "label": "Component",
            "properties": [
                {
                    "description": "",
                    "name": "name",
                    "type": "STRING"
                },
                {
                    "description": "",
                    "name": "description",
                    "type": "STRING"
                }
            ],
            "searchFields": [
                {
                    "description": "Semantic search field for the component's name.",
                    "name": "name_textembedding",
                    "type": "TEXT_EMBEDDING",
           

Extracting entities from PDF: 100%|██████████| 4/4 [01:02<00:00, 15.52s/it]


================================ Human Message =================================

what sequence of components depend on silicon wafers?
================================== Ai Message ==================================
Tool Calls:
  node_search (call_mhh8tLl0WbvwXxEyrS7zBxKN)
 Call ID: call_mhh8tLl0WbvwXxEyrS7zBxKN
  Args:
    search_query: silicon wafers
    top_k: 5
    search_config: {'search_type': 'SEMANTIC', 'node_label': 'Component', 'search_prop': 'name'}
================================= Tool Message =================================
Name: node_search

[
    {
        "id": "N26",
        "name": "Wafer",
        "description": "Silicon wafers are the basic building block for chip production. To produce them, a furnace forms a cylinder of silicon (or other semiconducting materials), which is then cut into disc-shaped wafers. These wafers are then processed, split and packaged into individual chips. Most wafers are made purely of silicon or another material, but others have more 

In [8]:
graphrag.agent("can you describe what gpus do?")

================================ Human Message =================================

can you describe what gpus do?
================================== Ai Message ==================================
Tool Calls:
  node_search (call_9MynE0nfm3RyToc5AUE2lxaN)
 Call ID: call_9MynE0nfm3RyToc5AUE2lxaN
  Args:
    search_config: {'search_type': 'SEMANTIC', 'node_label': 'Component', 'search_prop': 'name'}
    search_query: GPU
================================= Tool Message =================================
Name: node_search

[
    {
        "id": "N2",
        "name": "Logic chip design: Discrete GPUs",
        "description": "Discrete graphics processing units (\"GPUs\") have long been used for graphics processing (for example, in video game consoles) and in the last decade have become the most used chip for training artificial intelligence algorithms. The United States monopolizes the design market for GPUs, including standalone \"discrete GPUs,\" the most powerful GPUs.",
        "search_score"

In [9]:
graphrag.agent("what components have the most input?")

================================ Human Message =================================

what components have the most input?
================================== Ai Message ==================================
Tool Calls:
  aggregate (call_EuutU8EunlnGNYqhEhICtNUB)
 Call ID: call_EuutU8EunlnGNYqhEhICtNUB
  Args:
    agg_instructions: Aggregate the components based on the number of INPUT_TO relationships they have, and return the components with the most INPUT_TO relationships.
Running Query:
MATCH (c:Component)-[:INPUT_TO]->(:Component)
WITH c, COUNT(*) AS inputToCount
RETURN c.id, c.name, inputToCount
ORDER BY inputToCount DESC
================================= Tool Message =================================
Name: aggregate

[
    {
        "c.id": "N91",
        "c.name": "Electronic gases",
        "inputToCount": 5
    },
    {
        "c.id": "N59",
        "c.name": "Wafer and photomask handling",
        "inputToCount": 4
    },
    {
        "c.id": "N60",
        "c.name": "Process contr

In [7]:
# export and look at graph schema
graphrag.schema.export("graphrag-schema.json")

[Schema] Schema successfully exported to graphrag-schema.json


## Clean up

In [2]:
# drop everything in the graph
from neo4j import GraphDatabase

db_client = GraphDatabase.driver(uri, auth=(username, password))
with db_client.session() as session:
    # Delete all nodes and relationships
    session.run("MATCH (n) DETACH DELETE n")

    # Drop constraints
    session.run("CALL apoc.schema.assert({},{},true) YIELD label, key RETURN *")

    # Retrieve and drop search indexes
    result = session.run("""
        SHOW INDEXES YIELD name, type
        WHERE type IN ["FULLTEXT", "VECTOR"]
        RETURN name
    """)
    for record in result:
        index_name = record["name"]
        session.run(f"DROP INDEX {index_name} IF EXISTS")